In [ ]:
# Raw Package
import numpy as np
import pandas as pd

import APIBackend as back

In [ ]:

stock = back.stock()
# stock.ticker
ticker = '^GSPC' ### You Can Change the Ticker
stock_data = stock.getDailyStockInfo(ticker=ticker)




In [ ]:
stock.plot_close_stock(stock_data)
stock.trim_data(stock_data, month=5, year=2019)

In [ ]:
stock.getETSPlot()

In [ ]:
stock.plotDiagnostics()

In [ ]:
import statsmodels.api as sm
######################## Modeler Specifications ########################## You Can Change This Figures########################
ar = 2
ma = 3
###########################

stock.ARMA(ar,ma)



In [ ]:
stock.plotARMAFit()


In [ ]:
stock.forecastPlot()

In [ ]:
stock.saveDataFrame(save = True)

In [ ]:
######################## Modeler Specifications ########################## You Can Change This Figures########################
ar = [1]
ma = [12]

###########################

stock.ARIMA(ar,ma)



In [ ]:
# fig = plt.figure(figsize=(20,12))
# ax1 = fig.add_subplot(211)
# # fig = sm.graphics.tsa.plot_acf(stock_data.Close.dropna(),lags=24,ax=ax1)
# fig = sm.graphics.tsa.plot_acf(results.resid,lags=24,ax=ax1)
# ax2 = fig.add_subplot(212)
# # fig = sm.graphics.tsa.plot_pacf(stock_data.Close.dropna(),lags=24,ax=ax2)
# fig = sm.graphics.tsa.plot_pacf(results.resid,lags=24,ax=ax2)



In [ ]:
stock.forecastPlot()

In [ ]:
stock.saveDataFrame(save = True)